L'obiettivo della visualizzazione consiste nel verificare se il numero di score postoperatori fisici migliorati o peggiorati è influenzato dall'età del paziente

In [3]:
#importazione delle librerie
import pandas as pd
import plotly.io as pio
import plotly.graph_objs as go

Importazione dei dati

In [4]:
dataset= pd.read_csv("./compitino_dataviz_2021_2022.csv")
dataset.head()

,nome_modulistica,score_preop,score_postop,condizioni_meteo,sesso,anni_ricovero,temperatura_media,umidita_media,HI > 27,natural_light
0,SF12 PhysicalScore,27.60,37.40,poco nuvoloso,F,70,20.0,69.0,False,True
1,SF12 MentalScore,54.36,58.46,poco nuvoloso,F,70,20.0,69.0,False,True
2,SF12 PhysicalScore,28.15,38.25,poco nuvoloso,M,69,18.0,65.0,False,True
3,SF12 MentalScore,57.02,64.62,poco nuvoloso,M,69,18.0,65.0,False,True
4,SF12 PhysicalScore,31.57,55.67,poco nuvoloso,M,70,23.0,74.0,False,True


Calcolo colonne aggiuntive utili alla visualizzazione

In [5]:
dataset["delta_score"] = dataset["score_postop"] - dataset["score_preop"] 
dataset["peggiorato"] = dataset["delta_score"] < 0
dataset["pshysical_score"] = dataset["nome_modulistica"] == "SF12 PhysicalScore"

In [6]:

dataset["gruppo_eta"] = pd.qcut(dataset["anni_ricovero"], 6, labels=[
                                "(24, 58]", "(58, 65]", "(65, 69]", "(69, 74]", "(74, 79]", "(79, 97]"])
dataset["gruppo_eta"] = dataset["gruppo_eta"].astype(str)


Creazione di classi di età con numero di osservazioni omogeneo

In [7]:
#creazione di 6 intervalli di età
dataset["gruppo_eta"] = pd.qcut(dataset["anni_ricovero"], 6, labels=[
                                "(24, 58]", "(58, 65]", "(65, 69]", "(69, 74]", "(74, 79]", "(79, 97]"])
#conversione della colonna appena creata in stringa, per comodità 
dataset["gruppo_eta"] = dataset["gruppo_eta"].astype(str)


Calcolo per ogni classe d'età del numero punteggi fisici migliorati e peggiorati

In [8]:
#creazione nuovo dataframe
dataframe = pd.DataFrame({"gruppo_eta":[], "peggiorati": [], "migliorati":[]})
#ciclo for per ogni classe d'età
for item in dataset["gruppo_eta"].unique():
  #query per identificare i record con punteggio peggiorato per la classe d'età in esame
  queryStringPegg = "gruppo_eta == '"+str(item)+"' and peggiorato == 1 and pshysical_score == 1"
  #query per identificare i record con punteggio migliorato per la classe d'età in esame
  queryStringMigl = "gruppo_eta == '"+str(item)+"' and peggiorato == 0 and pshysical_score == 1"

  #query
  resPeggiorati = dataset.query(queryStringPegg)
  resMigliorati = dataset.query(queryStringMigl)
  #inserimento nel dataframe del numero di migliorati o peggiorati per la classe d'età in esame
  dataframe.loc[len(dataframe.index)] = [str(item), len(resPeggiorati), len(resMigliorati)]
#ordinamento delle classi d'età
dataframe = dataframe.sort_values("gruppo_eta")
#calcolo della percentuale di migliorati e peggiorati rispetto al totale di migliorati e peggiorati
dataframe["peggiorati"] =- (dataframe["peggiorati"]/ dataframe["peggiorati"].sum())
dataframe["migliorati"] = dataframe["migliorati"]/ dataframe["migliorati"].sum()
dataframe.head()


,gruppo_eta,peggiorati,migliorati
5,"(24, 58]",-0.150538,0.199079
4,"(58, 65]",-0.204301,0.154200
1,"(65, 69]",-0.107527,0.163406
0,"(69, 74]",-0.182796,0.176064
2,"(74, 79]",-0.225806,0.162255


Creazione della visualizzazione a piramide relativa alla percentuale di migliorati e peggiorati per ogni classe di età

In [9]:

#creazione finestra grafico
fig = go.Figure()
#creazione layout grafico con nome degli assi, titolo del grafico, etichette dei dati degli assi e grandezza del font
fig.update_layout(go.Layout(plot_bgcolor='rgba(0,0,0,0)',
                            yaxis=go.layout.YAxis(title='Classe di età'),
                            xaxis=go.layout.XAxis(
                                range=[-0.5, 0.5],
                                tickvals=[-0.5, -0.25, 0, 0.25, 0.5],
                                ticktext=[50, 25, 0, 25, 50],
                                title='Percentuale (%)'),
                            barmode='overlay',
                            # title="",
                            bargap=0.1,
                            font=dict(
                                size=14,
                            )

                            ))
#creazione della porzione di piramide relativo alla percentuale di peggiorati
fig.add_trace(go.Bar(y=dataframe["gruppo_eta"].to_list(), x=dataframe["peggiorati"],
                     orientation='h',
                     name='Peggiorati',
                     text=(dataframe["peggiorati"] *
                           100).abs().round(2).to_list(),
                     hoverinfo='skip',

                     marker=dict(color='#F4B942'),
                     textposition='auto',
                     ),)
#creazione della porzione di piramide relativo alla percentuale di migliorati
fig.add_trace(go.Bar(y=dataframe["gruppo_eta"].to_list(), x=dataframe["migliorati"],
                     orientation='h',
                     text=(dataframe["migliorati"] *
                           100).abs().round(2).to_list(),
                     name='Migliorati',
                     hoverinfo='skip',
                     marker=dict(color='#4059AD'),
                     textposition='auto',
                     ))

#visualizzazione grafico
fig.show()

Salvataggio della visualizzazione come immagine

In [10]:
#salvataggio della visualizzazione come file png con larghezza di 1700px
fig.write_image("pyramid.png", scale=1, width= 1700 )